In [1]:
import os
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py

In [2]:
## index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

urls = [
    "https://www.ai-jason.com/learning-ai/how-to-reduce-llm-cost",
    "https://www.ai-jason.com/learning-ai/gpt5-llm",
    "https://www.ai-jason.com/learning-ai/how-to-build-ai-agent-tutorial-3"
]

# doc = [FireCrawlLoader(api_key="fc-75d116f86f7d4eceb112c6c44d624508", url=urls, mode='scrape').load() for url in urls]

doc = []
for url in urls:
    loader = FireCrawlLoader(api_key="fc-75d116f86f7d4eceb112c6c44d624508", url=url, mode='scrape')
    # Try catch block to handle potential errors for each URL
    try:
        loaded_docs = loader.load()
        doc.append(loaded_docs)
    except Exception as e:
        print(f"Error loading {url}: {e}")


# Split documents
docs_list = [item for sublist in doc for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)

doc_splitter = text_splitter.split_documents(docs_list)

# Filter out complex metadata and ensure proper document formatting 
filtered_doc = []

# Correcting the variable name from 'doc_spilites' to 'doc_splitter'
for doc in doc_splitter:
    # Ensure the doc is an instance of document and has a metadata attribute
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# add to vectorDB
embedding = GPT4AllEmbeddings(model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf", gpt4all_kwargs = {'allow_download': 'True'})

vectorstore = Chroma.from_documents(
    documents = filtered_doc,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

# LLM
local_llm = 'llama3'
llm = ChatOllama(
    model=local_llm,
    format="json",
    temperature=0
)

# Prompt
prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    user
    Question: {question}
    Context: {context}
    Answer: assistant""",
    input_variables=["question", "context"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Conversation loop
while True:
    question = input("You:")
    if question.lower() in ['exit', 'quit']:
        break

    docs = retriever.invoke(question)
    context = format_docs(docs)
    generation = rag_chain.invoke({"context": context, "question": question})
    print("Assistant:", generation)


Assistant: {  }
Assistant: {  }
Assistant: { }


ValueError: text must not be None or empty

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the ChatOllama model
llm = ChatOllama(model=local_llm, temperature=0)

# Prompt template for the conversation
prompt_template = PromptTemplate(
    template="""You are an assistant for question-answering tasks. 
    Answer the following question concisely in three sentences or less.
    Question: {question}
    Answer:""",
    input_variables=["question"]
)

# Chain for processing the input and generating responses
def generate_response(question):
    prompt = prompt_template.format(question=question)
    response = llm.invoke(prompt)
    return response

# Conversation loop
while True:
    question = input("You: ")
    if question.lower() in ['exit', 'quit']:
        break
    response = generate_response(question)
    print("Assistant:", response)


Assistant: content="Nice to meet you, Haris! It's great that you're reaching out and saying hello. How can I assist you today?" response_metadata={'model': 'llama3', 'created_at': '2024-07-11T11:03:49.774491Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1995281959, 'load_duration': 33217042, 'prompt_eval_count': 50, 'prompt_eval_duration': 1078794000, 'eval_count': 28, 'eval_duration': 878849000} id='run-7e0fc909-acd7-445e-902d-9b02efba5e46-0'
Assistant: content='I\'m happy to help! However, it seems like you\'ve asked a rather... enthusiastic "wow" without any specific question or topic in mind. If you\'d like to ask something more concrete, I\'d be delighted to assist you with an answer!' response_metadata={'model': 'llama3', 'created_at': '2024-07-11T11:03:54.259679Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2083350584, 'load_duration': 39845667, 'prompt_eval_